# Getting Started with cntmosaic
In this guide, we will walk through installing and using the `cntmosaic` package to process data, run simulation and visualize results.

Current model supports fine integer age for participants, contacts and population. 

## 🔧 Installation
You can install `cntmosaic` from GitHub:

In [1]:
'''

!git clone https://github.com/ShozenD/cntmosaic.git
%cd cntmosaic
!pip install .

'''

'\n\n!git clone https://github.com/ShozenD/cntmosaic.git\n%cd cntmosaic\n!pip install .\n\n'

## 📦 Imports

In [2]:
from cntmosaic.dataloader import CoordToColumns, MergedLoader, RawLoader
from cntmosaic.models import restr_BRCfine
import pandas as pd
import numpy as np
from numpyro import distributions as dist


## 📁 Load Example Data

We will use the famous POLYMOD data as an example, you can get the csv files from https://zenodo.org/records/3746312#.Xo85CcgzZPY

Here we will show an interface similar to Socialmixr, a dataframe for participant, a dataframe for contacts and optionally, a dataframe for population. 

In [3]:
cnt = pd.read_csv('2008_Mossong_POLYMOD_contact_common.csv')
part = pd.read_csv('2008_Mossong_POLYMOD_participant_common.csv')

The model requires columns containing the participant age, the contact age, the id variable that connects the two dataframe. Other optional input include the contact count `y`. By default, each row represents one contact. Population characteristics can also be set, which will be demonstrated later.

`CoordToColumn` class can map the required columns to the model's need without changing the column name. Simply pass in the corresponding column name as shown below:

In [4]:
mapper = CoordToColumns(age_part='part_age', 
                        age_cnt='cnt_age_exact', 
                        id_var='part_id')

Afterwards, we can call the dataloader. It will merge by `id_var`. Use `data.raw_df` to inspect merged dataframe. Direct changes to `raw_df` are allowed. Then, use the $load$ function to prepare the dataloader to model use.

In [5]:
data = RawLoader(part, cnt, mapper)
data.raw_df = data.raw_df[data.raw_df.part_age < 85]
data.raw_df = data.raw_df[data.raw_df.cnt_age_exact < 85]
data.load()

Alternatively, if a merged panel form data is already available, $MergedLoader$ can be used. Additionally, if information about the population is available, we can pass them into $CoordToColumn$ and dataloader too. If population information is not available, uniform age distribution will be assumed. 

In [6]:
# Assume merged dataframe
merged_df = data.raw_df
# Assume population dataframe containing the population size for each age
pop = pd.DataFrame()
pop['pop_age'] = np.arange(0, 85, dtype=int)
pop['pop_size'] = 1
mapper = CoordToColumns(age_part='part_age', 
                        age_cnt='cnt_age_exact', 
                        id_var='part_id',
                        population_age='pop_age',
                        population_size='pop_size')
data = MergedLoader(merged_df, mapper, pop=pop)
data.load()


## 🧱 Call the Model

Both interface will generate a loaded dataloader object. We can pass this dataloader object to our model.

In [7]:
model = restr_BRCfine(data)

new model instantiated, please check default hyperparameters


Default parameters can be modified easily. `M` indicates the number of basis functions used to approximate the Gaussian Process, `C` is the boundary factor. We recommend keeping them unchanged. `grid-type` supports 'age-age' and 'diff-age'. If you suspect the contact dynamics have dependence on the age difference between participants and contacts, 'diff-age' would be more suitable. Additional offsets to contact intensity is possible via `offset`. It needs to be of the same dimension as the age grid.

The priors are numpryo distributions. beta0 is the baseline parameter, alpha is the magnitude and rho is the lengthscales. Please not rho is 2-D. Example below shows how to modify them.

In [8]:
print(model.params)

M: [30, 30]
C: [1.5, 1.5]
grid_type: age-age
likelihood: negbin
offset: None
prior:
beta0:<numpyro.distributions.continuous.Normal object at 0x700e710b1e80>
batch_shape:()
event_shape:()
loc:0.0
mean:0.0
scale:10.0
variance:100.0
alpha:<numpyro.distributions.continuous.InverseGamma object at 0x700e70af59a0>
batch_shape:()
concentration:5
event_shape:()
mean:1.25
rate:5
variance:0.5208333134651184
rho:<numpyro.distributions.distribution.ExpandedDistribution object at 0x700e70af56a0>
batch_shape:(2,)
event_shape:()
mean:[1.25 1.25]
variance:[0.5208333 0.5208333]


In [9]:
model.params.grid_type = 'diff-age'

In [10]:
model.params.prior['rho'] = dist.InverseGamma(5, 5).expand([2])
model.params.prior['alpha'] = dist.InverseGamma(5, 5)

We can generate samples from these priors.

In [ ]:
samples = model.prior_sampler('beta0', num_samples=10, seed=42)

Now we can run MCMC simulation.

In [11]:
model.run_inference_mcmc(rng_key=0,
    num_samples = 1000,
    num_warmup = 500,
    num_chains = 2)

/home/yiminglin/Downloads/cntmosaic/cntmosaic/models/_inference.py:28: UserWarning: There are not enough devices to run parallel chains: expected 2 but got 1. Chains will be drawn sequentially. If you are running MCMC in CPU, consider using `numpyro.set_host_device_count(2)` at the beginning of your program. You can double-check how many devices are available in your system using `jax.local_device_count()`.
  mcmc = MCMC(
sample: 100%|██████████| 1500/1500 [57:48<00:00,  2.31s/it, 511 steps of size 6.57e-03. acc. prob=0.93]  


Number of divergences: 5


It is recommended to save the model after MCMC is done.

In [ ]:
import pickle
with open('diff-age.pkl', 'wb') as f:
    pickle.dump(model, f)


## 📊 Visualize Model Outputs

We can call the summariser and visualiser to see the posterior contact intensity and contact rate.

In [17]:
from cntmosaic.analysis import ModelSummariserMCMC, ModelVisualiser
MSM = ModelSummariserMCMC(model)
visual = ModelVisualiser(MSM)

In [18]:
visual.plot_cint()

alt.Chart(...)

In [19]:
visual.plot_rate()

alt.Chart(...)

## 📚 Further Exploration
We can also extract posterior draws for each of our variables in the `post` attribute. Posterior contact intensity is stored in `post_cint`. 

In [15]:
MSM.post.keys()

dict_keys(['baseline', 'beta', 'beta0', 'inv_disp', 'log_cint', 'log_rate', 'rho'])

In [20]:
MSM.post_cint['general'].shape

(2000, 85, 85)